In [1]:
import ast
import json
from blablador import Models, Completions, ChatCompletions, TokenCount
from config import API_KEY, assistant, user, system
from utils import *


In [2]:
f = open('en.jsonl')
data = json.load(f)
f.close()
print(len(data))

5


In [3]:
# Retrieve available models
models = Models(api_key=API_KEY).get_model_ids()
for i in models:
    print(i)


1 - Mistral-7B-Instruct-v0.3 - the best option in general - fast and good (If it fails to respond then choose another model)
10 Mistral-Nemo-Instruct-2407 - New Mistral Nemo - give it a try
2 - Mixtral-8x7B-Instruct-v0.1 Slower with higher quality
3 - starcoder2-15b - A model for programming
4 - Cosmosage V3.1 - Answers your Cosmology and Astronomy questions (new version September 2024)
5 - GritLM-7B - For Chat AND Text Embeddings
6 - Llama3.1-8B-Instruct - A good model from META (update July 2024)
alias-code
alias-cosmosage
alias-embeddings
alias-fast
alias-fast-experimental
alias-large
alias-llama3-small
gpt-3.5-turbo
text-davinci-003
text-embedding-ada-002


## Experiments with promting

In [16]:
q_for_testing = "A car initially at rest accelerates in a straight line at 3 m/s\u00b2. What will be its speed after 2 seconds? Give just the number of correct answer without explanationsA: 0 m/s, B: 5 m/s, C: 3 m/s, D: 6 m/s, E: 2 m/s"
text_for_prompt = "Your task is to provide a direct answer to my question, selecting from four options.Please give only the letter corresponding to the correct answer without any explanations or additional information."

### Ordinary Full-reply, no promts

In [13]:
# Generate chat completions
completion = ChatCompletions(api_key=API_KEY, model=models[6])
response = completion.get_completion([{"role":"user", "content":q_for_testing}])
# Returns a JSON string
#print(ast.literal_eval(response)['model'])
print(ast.literal_eval(response)["choices"][0]["message"])


{'role': 'assistant', 'content': '## Step 1: Recall the formula for calculating speed under constant acceleration.\nThe formula to calculate the final speed (v) of an object under constant acceleration (a) is given by v = u + at, where u is the initial speed, a is the acceleration, and t is the time.\n\n## Step 2: Identify the given values.\nThe initial speed (u) of the car is 0 m/s since it is initially at rest, the acceleration (a) is 3'}


### Role: user, Promt: .., Content: ...

In [17]:
# Generate chat completions
completion = ChatCompletions(api_key=API_KEY, model=models[6])
response = completion.get_completion([{"role":"user", "promt":text_for_prompt ,"content":q_for_testing},])
# Returns a JSON string

print(ast.literal_eval(response)["choices"][0]["message"])
#{'role': 'assistant', 'content': "I'm not capable of experiencing emotions or having a physical body, but...




{'role': 'assistant', 'content': 'D'}


### Promt in the content

In [18]:
# Generate chat completions
completion = ChatCompletions(api_key=API_KEY, model=models[6])
response = completion.get_completion([{"role":"user","content":q_for_testing + text_for_prompt}])
# Returns a JSON string

print(ast.literal_eval(response)["choices"][0]["message"])


{'role': 'assistant', 'content': 'D'}


### Using role: system message. No examples

In [19]:
# Generate chat completions
completion = ChatCompletions(api_key=API_KEY, model=models[6])
response = completion.get_completion([{"role":"system", "content": text_for_prompt},
                                      {"role":"user", "content":q_for_testing}
                                      ])
# Returns a JSON string

print(ast.literal_eval(response)["choices"][0]["message"])


{'role': 'assistant', 'content': 'D'}


### Few-shot promting. User-assistant messages

In [20]:
completion = ChatCompletions(api_key=API_KEY, model=models[6])
response = completion.get_completion(messages=[
    user("You’re a highly knowledgeable physics tutor. For each message, give only the letter of the correct answer without any explanations or additional information."),
    user("A ball rolls down a slope and accelerates uniformly at 2 m/s². If it starts from rest, what will be its speed after 3 seconds? A. 3 m/s, B. 4 m/s, C. 5 m/s, D. 6 m/s, E. 7 m/s"),
    assistant("D"),
    user("A cyclist accelerates uniformly from rest to a speed of 10 m/s in 5 seconds. What is their acceleration? A. 1 m/s², B. 2 m/s², C. 3 m/s², D. 4 m/s², E. 5 m/s²r"),
    assistant("B"),
    user("A rocket accelerates from rest at a constant rate of 6 m/s². What speed will it reach after 4 seconds? A. 12 m/s, B. 18 m/s, C. 24 m/s, D. 30 m/s, E. 36 m/s"),
    assistant("C"),
    user(q_for_testing),
    ])
print(ast.literal_eval(response)["choices"][0]["message"])

{'role': 'assistant', 'content': 'D'}


## Running model with promts

In [6]:
ground_truth_llama, responses_llama = run_model_promting(path_to_questions = 'en.jsonl', path_to_answers = 'responses/responses_test.txt', model_id = 6,start_range = 0, end_range = 3, N_samples = 20)

Model name: 6 - Llama3.1-8B-Instruct - A good model from META (update July 2024)
